In [1]:
import pandas as pd
import numpy as np
import re

# For scraping
import requests
from bs4 import BeautifulSoup
import json

# Increase columns and rows for display to have a better view of the dataframe
pd.set_option("display.max_columns", 100)
pd.set_option("display.min_rows", 100)

### Residential monthly price index
- Source: full dataset from https://ec.europa.eu/eurostat/databrowser/view/STS_COPI_M/default/table?lang=en&category=sts.sts_cons.sts_cons_pri

In [2]:
# Read raw dataset
rpi_df = pd.read_csv('constr_price_cost_monthly.csv')
rpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17905 entries, 0 to 17904
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     17905 non-null  object 
 1   LAST UPDATE  17905 non-null  object 
 2   freq         17905 non-null  object 
 3   indic_bt     17905 non-null  object 
 4   cpa2_1       17905 non-null  object 
 5   s_adj        17905 non-null  object 
 6   unit         17905 non-null  object 
 7   geo          17905 non-null  object 
 8   TIME_PERIOD  17905 non-null  object 
 9   OBS_VALUE    17905 non-null  float64
 10  OBS_FLAG     9341 non-null   object 
dtypes: float64(1), object(10)
memory usage: 1.5+ MB


In [3]:
# Select relevant features
rpi_df = rpi_df[['TIME_PERIOD','geo','indic_bt','unit','OBS_VALUE']]

# Originally with different units and indic_bt so firt pivot and join column levels name 
rpi_df = rpi_df.pivot_table('OBS_VALUE',['TIME_PERIOD','geo'],['unit','indic_bt'])
rpi_df.columns = rpi_df.columns.map('_'.join)
rpi_df.reset_index(inplace=True)

#Transform 'Time period' to datetime with proper format so incomplete yeas can be filtered by date and rearrange order
rpi_df['TIME_PERIOD'] = pd.to_datetime(rpi_df['TIME_PERIOD'], format='%Y-%m')
rpi_df = rpi_df[['TIME_PERIOD','geo','I15_CSTO','PCH_PRE_CSTO','PCH_SM_CSTO']]
rpi_df = rpi_df[(rpi_df['TIME_PERIOD']>='2001-01-01')&(rpi_df['TIME_PERIOD']<='2022-12-01')]

#removing countries with incomplete data to aboid NaNs
rpi_df = rpi_df[(rpi_df['geo']!='IT')&(rpi_df['geo']!='TR')]

# Renaming features for better understanding
rpi_df.rename(columns={
    'TIME_PERIOD': 'month',
    'geo': 'country',
    'I15_CSTO': 'residential_price_index',
    'PCH_PRE_CSTO': 'perc_change_prev',
    'PCH_SM_CSTO':'perc_change_yoy'
}, inplace=True)
rpi_df.reset_index(drop=True, inplace=True)

rpi_df

,month,country,residential_price_index,perc_change_prev,perc_change_yoy
0,2001-01-01,AT,68.0,0.3,2.4
1,2001-01-01,ES,70.1,1.2,3.7
2,2001-01-01,FI,73.1,0.1,2.8
3,2001-01-01,IE,73.3,2.9,22.6
4,2001-01-01,LT,62.8,-1.7,0.6
5,2001-01-01,NL,76.4,1.2,3.9
6,2001-01-01,NO,60.1,1.0,5.3
7,2001-01-01,PL,82.4,0.7,4.8
8,2001-01-01,PT,74.8,-1.7,-2.2
9,2001-02-01,AT,68.0,0.0,2.3


In [4]:
rpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2376 entries, 0 to 2375
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   month                    2376 non-null   datetime64[ns]
 1   country                  2376 non-null   object        
 2   residential_price_index  2376 non-null   float64       
 3   perc_change_prev         2376 non-null   float64       
 4   perc_change_yoy          2376 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 92.9+ KB


In [5]:
# rpi_df.to_csv('res_price_index.csv', index=False)

### New Dwellings annual (NDA)

- Source:
    - NDA02.csv (https://data.cso.ie/table/NDA02)
    - NDA07.csv (https://data.cso.ie/table/NDA07)
    - NDA08.csv (https://data.cso.ie/table/NDA08)


In [6]:
# Define function to read multiple csv with arguments
def readcsv(args):
    return pd.read_csv(args, skiprows=1, header=None)

nda_df = pd.concat(map(readcsv, ['NDA02.csv','NDA08.csv','NDA07.csv']), ignore_index=True)
nda_df = nda_df.iloc[:,[1,2,5,7]]

nda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       152 non-null    object 
 1   2       152 non-null    int64  
 2   5       152 non-null    object 
 3   7       152 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.9+ KB


In [7]:
# Renaming columns 
nda_df.rename(columns={
    1:'label', 
    2:'year', 
    5:'prop_type', 
    7:'value'
}, inplace=True)

# Renaming labels
label_dict = {
    'New Dwelling Completions':'new_dwelling',
    'Average New Dwelling Size':'avg_dwelling_size', 
    'Weight in Mix':'dwelling_type_perc',
    'Average New Dwelling Size Index (2016=100)':'avg_dwelling_size_index2016'
}
nda_df.replace({"label":label_dict}, inplace=True)

# Renaming propertie_type names
prop_type_dict = {
    'All house types':'all_types',
    'Single house':'single_hs', 
    'Scheme house':'scheme_hs',
    'Apartment':'apartment',
    'Ireland': 'all_types'
}
nda_df.replace({"prop_type":prop_type_dict}, inplace=True)

nda_df = nda_df.pivot_table('value', ['year', 'prop_type'], 'label')
nda_df.reset_index(drop=False, inplace=True)
nda_df = nda_df.rename_axis(None, axis=1)
nda_df.drop(['avg_dwelling_size_index2016'], axis=1, inplace=True)

# Property_type percent for all types is 100%
nda_df['dwelling_type_perc'] = nda_df['dwelling_type_perc'].fillna(100)

# No data for year 2023
nda_df.dropna(inplace=True)

nda_df

,year,prop_type,avg_dwelling_size,dwelling_type_perc,new_dwelling
0,2011,all_types,194.9,100.0,6994.0
1,2011,apartment,73.0,11.8,822.0
2,2011,scheme_hs,109.0,19.4,1358.0
3,2011,single_hs,240.0,68.8,4814.0
4,2012,all_types,200.8,100.0,4911.0
5,2012,apartment,65.0,9.1,446.0
6,2012,scheme_hs,116.0,19.6,964.0
7,2012,single_hs,241.0,71.3,3501.0
8,2013,all_types,196.1,100.0,4575.0
9,2013,apartment,74.0,10.3,473.0


In [8]:
# nda_df.to_csv('nda.csv', index=False)

### Daft.ie house prices by property type

- Source: Web scraping (Daft.ie)
- around 3 minutes to run

In [9]:
# URL of the page with pagination
url = 'https://www.daft.ie/property-for-sale/ireland?floorSize_from=10&floorSize_to=2500&spageSize=20&numBeds_from=1&numBeds_to=9&numBaths_from=1&numBaths_to=9&pageSize=20&propertyType=detached-houses&propertyType=apartments&propertyType=semi-detached-houses&from='

# Pages show 20 resuls per page, create a list to loop every 20 posts
pages = [*range(0,8501,20)]

# Empty list to store each loop
data = []

# Loop through all pages
for page in pages:  
    # Construct the URL of the current page
    page_url = url + str(page)
    
    # Send a GET request to the page URL
    response = requests.get(page_url)
    
    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for span in soup.find_all('span'):
        span.decompose()
    
    # Scrape data from the current page using BeautifulSoup methods
    price = [item.text for item in soup.find_all(attrs={'data-testid':'price'})]
    floor_area = [item.text for item in soup.find_all(attrs={'data-testid':'floor-area'})]
    beds = [item.text for item in soup.find_all(attrs={'data-testid':'beds'})]
    baths = [item.text for item in soup.find_all(attrs={'data-testid':'baths'})]
    property_type = [item.text for item in soup.find_all(attrs={'data-testid':'property-type'})]
    
    # Combine the scraped data into a list of dictionaries
    page_data = [{
        'price': price,
        'floor_area': floor_area,
        'beds': beds,
        'baths': baths,
        'property_type': property_type
    } 
        for price, floor_area, beds, baths, property_type in zip(price, floor_area, beds, baths, property_type)]

    # Add the page data to the list of all data
    data.extend(page_data)
    
# Create a Pandas DataFrame from the list of dictionaries
daft_df = pd.DataFrame(data)

# Print the DataFrame
daft_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7289 entries, 0 to 7288
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   price          7289 non-null   object
 1   floor_area     7289 non-null   object
 2   beds           7289 non-null   object
 3   baths          7289 non-null   object
 4   property_type  7289 non-null   object
dtypes: object(5)
memory usage: 284.9+ KB


In [10]:
# define a function to remove non-numeric characters from a string
def remove_nonnumeric_chars(s):
    return re.sub('[^0-9]', '', s)

# apply the function to the desired columns
daft_df['price'] = daft_df['price'].apply(remove_nonnumeric_chars).apply(pd.to_numeric)
daft_df['beds'] = daft_df['beds'].apply(remove_nonnumeric_chars).apply(pd.to_numeric)
daft_df['baths'] = daft_df['baths'].apply(remove_nonnumeric_chars).apply(pd.to_numeric)

# Remove rows whit NaN values in price due to previous applied function
daft_df = daft_df.dropna(subset=['price']).reset_index(drop=True)

# Split 'floor_area' into 'floor_area'(number) and 'floor_area_unit'(str)
daft_df[['floor_area','floor_area_unit']] = daft_df['floor_area'].str.split(' ',expand=True)

# Remove properties in different area unit (mostly not houses/apartments)
daft_df = daft_df.drop(daft_df[daft_df.floor_area_unit =='ac'].index)
# There were some listings (3) with non realistic prices 
daft_df = daft_df.drop(daft_df[daft_df.price < 100].index).reset_index(drop=True)

# Convert numeric columns to int
daft_df[['price','floor_area','beds','baths']] = daft_df[['price','floor_area','beds','baths']].astype('float')

# Add calculated column to show price per square meter
daft_df['price_sq_mt'] = round(daft_df['price']/daft_df['floor_area'])

In [11]:
daft_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7012 entries, 0 to 7011
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   price            7012 non-null   float64
 1   floor_area       7012 non-null   float64
 2   beds             7012 non-null   float64
 3   baths            7012 non-null   float64
 4   property_type    7012 non-null   object 
 5   floor_area_unit  7012 non-null   object 
 6   price_sq_mt      7012 non-null   float64
dtypes: float64(5), object(2)
memory usage: 383.6+ KB


In [12]:
# daft_df.to_csv('daft.csv', index=False)

### Sentiment analysis

- News titles from
    - The Journal
    - World Construction Today
    - Irish Time

### The Journal news

- Source: Web scraping (thejournal.ie)

In [13]:
# URL of the page with pagination
url = 'https://www.thejournal.ie/housing/news/page/'

# First 10 pages (around mar 2016)
pages = [*range(1,31)]

# Empty list to store each loop
data = []

# Loop through all pages
for page in pages:  
    # Construct the URL of the current page
    page_url = url + str(page)
    
    # Send a GET request to the page URL
    response = requests.get(page_url)
    
    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Scrape data from the current page using BeautifulSoup methods
    lines = [item.text for item in soup.find_all(attrs={'class':['title-redesign','excerpt-redesign']})]

    # Add the page data to the list of all data
    data.extend(lines)
    
# Create a Pandas DataFrame from the list of dictionaries
tj_df = pd.DataFrame(data)

# Print the DataFrame
tj_df

,0
0,"Green light for nine-storey, 245-bedroom 'lean..."
1,Dublin City Council stated that the scheme is ...
2,Dublin Fire Brigade to tell TDs that fire safe...
3,Traditional building methods have undergone a ...
4,O'Brien seeking to convert vacant office block...
5,The Housing Minister today confirmed that he w...
6,\n 'Not going back to hostels': The...
7,\n Tánaiste Micheál Martin said tha...
8,"House prices up 3.9% from last year, the lowes..."
9,House prices outside Dublin are rising faster ...


In [14]:
tj_df[0] = tj_df[0].replace('\n|\t|\r','',regex=True)
tj_df[0] = tj_df[0].map(str.strip)
tj_df[0]

0       Green light for nine-storey, 245-bedroom 'lean...
1       Dublin City Council stated that the scheme is ...
2       Dublin Fire Brigade to tell TDs that fire safe...
3       Traditional building methods have undergone a ...
4       O'Brien seeking to convert vacant office block...
5       The Housing Minister today confirmed that he w...
6       'Not going back to hostels': The Dublin tenant...
7       Tánaiste Micheál Martin said that the RTB woul...
8       House prices up 3.9% from last year, the lowes...
9       House prices outside Dublin are rising faster ...
10      Tory bill to abolish no-fault evictions in Eng...
11      The ban was part of the Conservatives’ 2019 el...
12      Poll: Do you know the Building Energy Rating (...
13                                    What’s your rating?
14      Micheál Martin: It's 'possible' Ireland can bu...
15      The Tánaiste says Ireland’s construction indus...
16      Analysis:                 Government and the C...
17      Mark C

In [15]:
# tj_df.to_csv('theJournal.csv', index=False)

### World Construction Today news

- Source: Web scraping (worldconstructiontoday.com)
- around 20 minutes to run

In [16]:
# URL of the page with pagination
url1 = 'https://www.worldconstructiontoday.com/page/'
url2 = '/?s=construction'

# Create a list to use as pages
pages = [*range(1,300)]
# pages = [1,2] # to test code

# Empty list to store each loop
data = []

# Loop through all pages
for page in pages:  
    # Construct the URL of the current page
    page_url = url1 + str(page) + url2
    
    # Send a GET request to the page URL
    response = requests.get(page_url)
    
    # Parse the HTML content of the response using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    for article in soup.find_all(attrs={'class':'td_block_inner'}):
        article.decompose()
    
    # Scrape data from the current page using BeautifulSoup methods
    lines = [item.text for item in soup.find_all(attrs={'class':['entry-title td-module-title','td-excerpt']})]

    # Add the page data to the list of all data
    data.extend(lines)
    
# Create a Pandas DataFrame from the list of dictionaries
wc_df = pd.DataFrame(data)

# Print the DataFrame
wc_df

,0
0,Top 10 Mistakes to Avoid on a Construction Site
1,\r\n Construction sites can...
2,Oldest Construction Examples Found In Saudi An...
3,\r\n Archaeologists have go...
4,Revolutionising the Construction Industry with...
5,\r\n In the ever-evolving w...
6,Staying Safe in Construction: Common Hazards a...
7,\r\n Construction is one of...
8,Construction Input Prices Show Favorable Trend...
9,\r\n \r\n \tAccording to an...


In [17]:
wc_df[0] = wc_df[0].replace('\n|\t|\r|','',regex=True)
wc_df[0] = wc_df[0].map(str.strip)
wc_df[0]

0         Top 10 Mistakes to Avoid on a Construction Site
1       Construction sites can be busy, complex, and s...
2       Oldest Construction Examples Found In Saudi An...
3       Archaeologists have gone on to describe what c...
4       Revolutionising the Construction Industry with...
5       In the ever-evolving world of construction, a ...
6       Staying Safe in Construction: Common Hazards a...
7       Construction is one of the most essential indu...
8       Construction Input Prices Show Favorable Trend...
9       According to an analysis of government data re...
10      Study Shows How Technology Transforms Construc...
11      A new study has gone on to highlight how techn...
12      Construction Investments Boost Solar Shading S...
13      In the past decade, the solar shading systems ...
14      UK Government To Standardize Modern Constructi...
15      The UK Government is planning to standardize m...
16      Construction Insights: The Future of Sustainab...
17      As our

In [18]:
# wc_df.to_csv('worldconstructiontoday.csv', index=False)

### Irish Times news

- Source: Web scraping (irishtimes.com)

In [19]:
# URL of the page with pagination
url1 = 'https://api.queryly.com/json.aspx?queryly_key=954cd8bb239245f9&query=house%20prices%20&endindex='
url2 = '&batchsize=100&callback=searchPage.resultcallback&showfaceted=true&extendeddatafields=creator,imageresizer,promo_image&timezoneoffset=-60'

# around 2018
article_index = [*range(0,801,100)]

# Empty dataframe to store each loop
articles = {'title':[],'description':[]}
it_df = pd.DataFrame(articles)

# Loop through all pages
for i in article_index:  
    # Construct the URL of the current page
    page_url = url1 + str(i) + url2
    
    # Send a GET request to the page URL
    response = requests.get(page_url)

    resp=response.text # couldn't parse to json some extra chars and line breaks
    resp2 = resp[42:-27] # remove extra chars
    resp_json = json.loads(resp2) # to dictionary
    items = resp_json['items'] # access articles (list of dictionaries)
    items_df = pd.DataFrame(items) #transform to dataframe
    
    loop_df = items_df[['title', 'description']]
    
    it_df = pd.concat([it_df, loop_df], ignore_index=True, sort=False)
    
it_df

,title,description
0,Housing crisis – a modest proposal,"Sir, – In view of the likely further windfall ..."
1,"Apple tax case begins, housing affordability, ...",Apple told the European Union’s highest court ...
2,Castleknock house prices: Six properties and w...,Where: 13 Castleknock Grove What: Three-bed se...
3,Dublin has third lowest level of low-cost hous...,Dublin has one of the lowest concentrations of...
4,Housing starts continue to rise despite hike i...,"Housing commencements, one of the strongest in..."
5,Cliff Taylor: Are Irish house prices set to fa...,House prices in March were 3.9 per cent ahead ...
6,House price inflation slows to 3.9% as interes...,House price inflation continues to slow in the...
7,What should we be aware of when purchasing a h...,If my family were to purchase a house privatel...
8,Housing crisis and land-price speculation,"Sir, – Further to Gerard Howlin’s article “Mor..."
9,Housing and the price of land,"Sir, – David Doran (Letters, May 15th) writes ..."


In [20]:
# it_df.to_csv('irish_times.csv', index=False)